# Información Geográfica y Machine Learning: un ejemplo de modelo de ayuda a la toma de decisiones en la expansión de alojamientos turísticos

En este Jupyter Notebook (versión Colab de Google) se va explicar, paso a paso, la elaboración de un modelo territorial orientado a la toma de decisiones en una operación de expansión de alojamientos turísticos.


## Jupyter Notebooks y Google Colab

Esta herramienta que estamos utilizando se llama **Jupyter Notebook**, en su versión cloud patrocinada y hospedada por Google, llamado **Google Colab** (o Colaboratory). Los Jupyter Notebooks son una herramienta muy arraigada en el ecosistema Python (aunque el modelo ha sido exportado a otros entornos, como JavaScript, por ejemplo, o R). Los objetivos de esta herramienta son básicamente dos:

- establecer un entorno **interactivo** de computación en Python que facilita la exploración de datos y código;
- tiene una orientación eminentemente didáctica, puesto que permite combinar celdas (como esta) de texto enriquecido para articular una explicación con otras de código Python ejecutable.

Esta versión Colab de Jupyter se ejecuta en máquinas en la nube de Google. Su uso es gratuíto y puede escalar, con pago por uso o demanda, a entornos más potentes si se precisa.

Cuando se arranca un Notebook Colab / Jupyter, éste arranca en una máquina de Google y ejecuta su código sobre un **kernel** Python, es decir, un intérprete de Python con el que podemos interaccionar, cargándole datos y definiendo operaciones sobre ellos. Por ejemplo, en la siguiente celda de código Python definimos dos variables en la memoria del kernel, **a** y **b**:

In [3]:
a = 1
b = 10

Y en la siguiente celda de código realizamos una operación interactiva con ellos:

In [5]:
a + b

11

Sin embargo, como el objetivo de la sesión es metodológico y no técnico, evitaremos en la medida de lo posible el código Python encapsulandolo en una librería escrita en un módulo aparte. Dicho módulo está disponible en el repositorio de origen de este Notebook [https://github.com/malkab/turismo_geo_ml/](https://github.com/malkab/turismo_geo_ml/) en la carpeta D: XXX.